# Motivation
论文原文：[《Revisiting Deep Learning Models for Tabular Data》](https://arxiv.org/abs/2106.11959)

参考学习地址：[如何使用Transformer处理分类任务（表格数据、详细代码讲解）](https://zhuanlan.zhihu.com/p/685001211)

Feature Tokenizer + Transformer 作为Transformer架构兼容Tabular领域数据的简单改造。改造内容如其名，将原始输入向量化后，作为输入放入一系列transformer层（因此每个Transformer层作用于一个对象的特征层面，类比AutoInt），通过添加[CLS] token用于预测。

 <img style="display: block; margin: 0 auto;" src="../../../assets/images/ft-transformer.png" width = "600" height = "550" alt="FT-Transformer" align=center />

# Feature Tokenizer
转换输入特征 $x$ 为嵌入表征 $T\in \R^{k\times d}$，对一个给定的特征 $x_j$ 其向量表征为
$$
\begin{align}
T_j=b_j+f_j(x_j)\in \R^d \text{    } f_j:X_j \rightarrow \R^d
\end{align}
$$
其中 $f_j^{(num)}$ 由和 $W_j^{(num)}\in \R^d$ 逐元素相乘获取，$f_j^{(cat)}$ 为 lookup table $W_j^{(cat)}\in \R^{S_j\times d}$。综上：
$$
\begin{align}
T_j^{(num)}=b_j^{(num)}+x_j^{(num)}W_j^{(num)}& \in \R^d \\
T_j^{(cat)}=b_j^{(cat)}+e_j^TW_j^{(cat)}& \in \R^d \\
T=stack[T_1^{(num)}, ..., T_{k^{(num)}}^{(num)},T_1^{(cat)}, ..., T_{k^{(cat)}}^{(cat)}]& \in \R^{k\times d} \\
\end{align}
$$
其中 $e_j$ 为对应类别向量的one-hot表示。

## 类别偏置
处理分类特征时通常希望确保每个类别特征的嵌入向量唯一，也即假设数据是{'天气'：{'晴天'：0，'阴天'：1}，'月份'：{'一月'：0}}，虽然晴天和一月的输入都是0，但并不希望在Embedding过程中把他们映射到同一个向量上，使用偏移量之后{'天气'：{'晴天'：0，'阴天'：1}，'月份'：{'一月'：0+2}}，这个2就是偏移量，目标是确保模型可以正确地为每个类别特征学习到一个独立的嵌入表示。

# Transformer
在embedding前concat CLS token，$T_0=stack[[CLS], T]$。<font color="red">注意，FT-Transformer使用的PreNorm，同时移除了首层输入的Normalization</font>，且主要使用的是Encoder。
